In [2]:
from numpy import linalg as LA 
import scipy.linalg as la
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'scipy'

In [2]:
# Conversion parameters for effective mass atomic unit

m2ang = 1 / 1e-10 # Length
def get_conversion_parameters(effective_mass):
    # conversion parameters for effective mass atomic unit
    eV2eu = np.float64(1/27.211386245988)
    ang2eu  = np.float64(1.889725989) 
    amp2eu  = 1/np.float64(6.623618237*1e-3) 
    
   # return eV2eu, ang2eu, amp2eu
    return eV2eu, ang2eu, amp2eu

In [3]:
# 0D with inelastic scattering

def inelastic_scattering(em, H0, D0, D1, D2, hw1, hw2, label):
    
    eV2eu, ang2eu, amp2eu = get_conversion_parameters(em)
    
    
    # Parameters (eu)
    H0 = H0 * eV2eu
    Ef = 0 * eV2eu
    kT = 0.0025 * eV2eu
    dE = 0.0005 * eV2eu
    zplus = 1j * 1e-12 * eV2eu
    gamma = 0.1 * eV2eu

    I0 = 1/(2*np.pi)
    Dnu = eV2eu**2*np.array([D1,D2], dtype = float)
    Nph = 2
    hnu = np.array([hw1, hw2], dtype = float)

    Nhnu =1/(np.exp(dE*hnu/kT)-1)

    # Bias
    NV = 203
    VV = eV2eu * np.linspace(-0.51, 0.50, NV)
    dV = VV[1] - VV[0]


    II = np.zeros(NV)
    for iv in range(NV):
        V = VV[iv]
        mu1 = Ef
        mu2 = Ef-V
        U1 = -0.5*V
       # print ("U1=", U1/eV2eu)
        # Energy grid
        if V < 0:
            emin = mu1-(10*kT)-(10*dE)
            emax = mu2+(10*kT)+(10*dE)
        else:
            emin = mu2-(10*kT)-(10*dE)
            emax = mu1+(10*kT)+(10*dE)
        NE = int(round((emax - emin)/dE)) + 1
        E = np.linspace(emin, emax, NE)
        f1 = 1/(1+np.exp((E-mu1)/kT))
        f2 = 1/(1+np.exp((E-mu2)/kT))

        # Initial guess
        n = np.zeros(NE)
        p = np.zeros(NE)
        sigin1  = np.zeros(NE)
        sigout1 = np.zeros(NE)
        sigin2  = np.zeros(NE)
        sigout2 = np.zeros(NE)
        sigin  = np.zeros(NE)
        sigout = np.zeros(NE)
       
      #  plt.plot(U1)
       # plt.show()
        
        # Iterative solution of transport equation
        change =1 
        it = 1
        while change > 1e-5:
            for k in range(NE):
                sig1 = -1j * gamma/2
                gam1 = 1j * (sig1-np.conjugate(sig1))
                sig2 = -1j * gamma/2
                gam2 = 1j * (sig2-np.conjugate(sig2))

                sigin1[k] = f1[k] * gam1
                sigin2[k] = f2[k] * gam2
                sigout1[k] = (1-f1[k])*gam1
                sigout2[k] = (1-f2[k])*gam2
                gamp = sigin[k] + sigout[k]
                G = 1/((E[k]+zplus)-H0-U1-sig1-sig2+(1j*0.5*gamp))
                A = 1j * (G-np.conjugate(G))
                n[k] = np.real(G*((f1[k]*gam1 + f2[k]*gam2)+sigin[k])*np.conjugate(G))
                p[k] = A - n[k]

            siginnew = D0 * n
            sigoutnew = D0 * p

            for iph in range(Nph):
                inu = int(hnu[iph])
                if inu < NE:

                    zero = []
                    for i in range(inu):
                        zero.append(0)

                    # ne
                    ne = [n[i] for i in range(inu,NE)]
                    ne = np.array(ne+zero)

                    # na
                    na = [n[i] for i in range(0,NE-inu)]
                    na = np.array(zero+na)

                    # pe
                    pe = [p[i] for i in range(inu,NE)]
                    pe = np.array(pe+zero)

                    # pa
                    pa = [p[i] for i in range(0,NE-inu)]
                    pa = np.array(zero+pa)

                    siginnew = siginnew + ((Nhnu[iph]+1)*Dnu[iph]*ne) + (Nhnu[iph]*Dnu[iph]*na)
                    sigoutnew = sigoutnew + (Nhnu[iph]*Dnu[iph]*pe) + ((Nhnu[iph]+1)*Dnu[iph]*pa)

            change = sum(np.abs(siginnew-sigin))
            change = change + sum(np.absolute(sigoutnew-sigout))
            sigin = ((1-it)*sigin)+ (it*siginnew)
            sigout = ((1-it)*sigout) + (it*sigoutnew)
       # print(iv)
       # print(change)

        I1 = np.real((sigout1*n)-(sigin1*p))
        I1 = sum(I1)
        I2 = np.real((sigout2*n)-(sigin2*p))
        I2 = sum(I2)
        I3 = np.real((sigout*n)-(sigin*p))
        I3 = sum(I3)

        II[iv] = I2 * dE * I0
 #   print ('dv', dV)
    G1 = np.diff(II)/dV * eV2eu / amp2eu
    VG = VV[1:NV] / eV2eu
    IETS = np.diff(G1)/dV * eV2eu
    VETS = VV[2:NV] / eV2eu
    VV = VV / eV2eu
    II = II / amp2eu
    fig = plt.figure(figsize=[18.0,6.0])

In [4]:
ax1 = fig.add_subplot(1,3,1)
ax1.set_title("I vs. V")

ax2 = fig.add_subplot(1,3,2)
ax2.set_title("dI/dV vs. V")

ax3 = fig.add_subplot(1,3,3)
ax3.set_title(r"d$^{2}$I/dV$^{2}$ vs. V")

#inelastic_scattering(em, H0, D0, D0, D0, label = r'D$_{0}$=0.0')
inelastic_scattering(em, H0*0.8, D0, D1, D2, hw1, hw2, label = r'0.8H$_{0}$')
inelastic_scattering(em, H0*0.9, D0, D1, D2, hw1, hw2, label = r'0.9h$_{0}$')
inelastic_scattering(em, H0, D0, D1, D2, hw1, hw2, label = r'1h$_{0}$')
plt.legend()
plt.show()
    ax1.plot(VV,II, label = label)
    ax2.plot(VG,G1, label = label) 
    ax3.plot(VETS,IETS, label = label)

IndentationError: unexpected indent (<ipython-input-4-d0e5d0aa2ac3>, line 16)

In [ ]:
em = 1
H0 = 5
D0 = 0; D1 = 0.5; D2 = 0.7
hw1 = 150; hw2 = 550

In [ ]:
fig = plt.figure(figsize=[18.0,6.0])

ax1 = fig.add_subplot(1,3,1)
ax1.set_title("I vs. V")
#ax1.set_xlim([-0.5,0.5])
#ax1.set_ylim([-1e-8,1e-8])

ax2 = fig.add_subplot(1,3,2)
ax2.set_title("dI/dV vs. V")
#ax2.set_xlim([-0.6,0.6])
#ax2.set_ylim([1.53e-8,1.63e-8])

ax3 = fig.add_subplot(1,3,3)
ax3.set_title(r"d$^{2}$I/dV$^{2}$ vs. V")
#ax3.set_xlim([-0.5,0.5])
#ax3.set_ylim([-3e-8,3e-8])



#inelastic_scattering(em, H0, D0, D0, D0, label = r'D$_{0}$=0.0')
inelastic_scattering(em, H0, D0, D1, D2, hw1, hw2, label = r'D$_{1/2}$=0.5, 0.7')
plt.legend()
plt.show()fig = plt.figure(figsize=[18.0,6.0])

ax1 = fig.add_subplot(1,3,1)
ax1.set_title("I vs. V")

ax2 = fig.add_subplot(1,3,2)
ax2.set_title("dI/dV vs. V")

ax3 = fig.add_subplot(1,3,3)
ax3.set_title(r"d$^{2}$I/dV$^{2}$ vs. V")

#inelastic_scattering(em, H0, D0, D0, D0, label = r'D$_{0}$=0.0')
inelastic_scattering(em, H0*0.8, D0, D1, D2, hw1, hw2, label = r'0.8H$_{0}$')
inelastic_scattering(em, H0*0.9, D0, D1, D2, hw1, hw2, label = r'0.9h$_{0}$')
inelastic_scattering(em, H0, D0, D1, D2, hw1, hw2, label = r'1h$_{0}$')
plt.legend()
plt.show()